##### Module import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성

In [3]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

##### Data import

In [4]:
print(os.getcwd())

c:\Users\Owner\Desktop\윤태준\고용패널\고용패널 파이썬 분석\기업변수모형_근무시간_binary


In [5]:
df = pd.read_csv('2023_04_13_data_step.csv', encoding='cp949')

In [6]:
df_1 = df.copy()
df_1.columns

Index(['pid', 'majorcat', 'province', 'area', 'sex', 'age', 'a003', 'a010',
       'a038', 'a039',
       ...
       'company_satiG', 'supply', 'insurance', 'work_timeB', '주당근로일', '주당근로시간',
       '직장에서의복리후생제공여부', '보험가입여부', '직장만족도', '주당근로시간_binary'],
      dtype='object', length=108)

In [8]:
df_1.주당근로시간_binary.value_counts()

40시간이하    4758
40시간초과    1050
Name: 주당근로시간_binary, dtype: int64

##### 다변량 모델

#### 복지 모델

In [6]:
model_1 = sm.Logit.from_formula('''LongWork2 ~ C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_1.summary()

Optimization terminated successfully.
         Current function value: 0.544362
         Iterations 5
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5805
Method:                           MLE   Df Model:                            2
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                 0.06165
Time:                        21:03:51   Log-Likelihood:                -3161.7
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 6.229e-91
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -1.6740      0.055    -30.503      0.000      -1.782      -1.566
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     1.0717      0.069     15.474      0.000       0.936       1.207
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.6123      0.067      9.104      0.000       0.481       0.744
==================================================================================================================
"""

In [7]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_1 = pd.DataFrame(
    {
        "OR": model_1.params,
        "Lower CI": model_1.conf_int()[0],
        "Upper CI": model_1.conf_int()[1],
    }
)
odds_ratios_1 = np.exp(odds_ratios_1)
odds_ratios_1

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.187490,0.168370,0.208782
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.920222,2.549571,3.344757
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.844752,1.616896,2.104717


In [10]:
model_2 = sm.Logit.from_formula('''LongWork2 ~ C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음')) + 주당근로일 
    + C(주당근로시간_binary,Treatment('40시간초과'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_2.summary()

Optimization terminated successfully.
         Current function value: 0.542080
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5803
Method:                           MLE   Df Model:                            4
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                 0.06558
Time:                        21:14:12   Log-Likelihood:                -3148.4
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 2.424e-94
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -2.7046      0.223    -12.109      0.000      -3.142      -2.267
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]      1.0424      0.070     14.998      0.000       0.906       1.179
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]              0.5155      0.070      7.411      0.000       0.379       0.652
C(주당근로시간_binary, Treatment('40시간초과'))[T.40시간이하]     0.1609      0.084      1.909      0.056      -0.004       0.326
주당근로일                                               0.2016      0.041      4.957      0.000       0.122       0.281
===================================================================================================================
"""

In [11]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_2 = pd.DataFrame(
    {
        "OR": model_2.params,
        "Lower CI": model_2.conf_int()[0],
        "Upper CI": model_2.conf_int()[1],
    }
)
odds_ratios_2 = np.exp(odds_ratios_2)
odds_ratios_2

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.066896,0.043180,0.103640
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.835931,2.474771,3.249797
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.674411,1.461022,1.918967
"C(주당근로시간_binary, Treatment('40시간초과'))[T.40시간이하]",1.174557,0.995697,1.385547
주당근로일,1.223331,1.129606,1.324832


In [12]:
model_3 = sm.Logit.from_formula('''LongWork2 ~ 
    + C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))
    + 주당근로일 + C(주당근로시간_binary,Treatment('40시간초과'))
    + C(직장만족도,Treatment('그렇지않다'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_3.summary()

Optimization terminated successfully.
         Current function value: 0.524364
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5801
Method:                           MLE   Df Model:                            6
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                 0.09612
Time:                        21:19:37   Log-Likelihood:                -3045.5
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                1.186e-136
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -3.5866      0.242    -14.836      0.000      -4.060      -3.113
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]      0.9200      0.071     12.920      0.000       0.780       1.060
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]              0.5706      0.071      8.056      0.000       0.432       0.709
C(주당근로시간_binary, Treatment('40시간초과'))[T.40시간이하]     0.0922      0.086      1.068      0.285      -0.077       0.261
C(직장만족도, Treatment('그렇지않다'))[T.그렇다]                 1.1959      0.094     12.704      0.000       1.011       1.380
C(직장만족도, Treatment('그렇지않다'))[T.보통이다]                0.5418      0.098      5.502      0.000       0.349       0.735
주당근로일                                               0.2398      0.041      5.820      0.000       0.159       0.321
===================================================================================================================
"""

In [13]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3 = pd.DataFrame(
    {
        "OR": model_3.params,
        "Lower CI": model_3.conf_int()[0],
        "Upper CI": model_3.conf_int()[1],
    }
)
odds_ratios_3 = np.exp(odds_ratios_3)
odds_ratios_3

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.027692,0.017242,0.044477
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.509326,2.182457,2.885150
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.769345,1.540011,2.032831
"C(주당근로시간_binary, Treatment('40시간초과'))[T.40시간이하]",1.096602,0.925926,1.298739
"C(직장만족도, Treatment('그렇지않다'))[T.그렇다]",3.306614,2.749510,3.976597
"C(직장만족도, Treatment('그렇지않다'))[T.보통이다]",1.719137,1.417393,2.085118
주당근로일,1.271031,1.172414,1.377944
